# Image MEA
1. Setup a project
2. Import labels with MAL
3. Correct the labels in the editor
4. Upload labels and predictions for MEA
------ 

## Environment Setup
* Install dependencies

In [ ]:
!pip install labelbox==2.7b1 \
             requests \
             ndjson \
             scikit-image \
             PILLOW \
             tensorflow \
             opencv-python

In [1]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python && git checkout mea-dev
    !mv labelbox-python/examples/model_assisted_labeling/*.py .

* Import libraries

In [2]:
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, LabelingFrontend, MALPredictionImport
from image_model import predict, class_mappings, load_model
from io import BytesIO
from getpass import getpass
import uuid
import numpy as np
from PIL import Image
import requests
from collections import defaultdict
import ndjson
import os
from ndjson_utils import (
    create_boxes_ndjson, 
    create_polygon_ndjson, 
    create_mask_ndjson, 
    create_point_ndjson
)
from labelbox.data.metrics.iou import data_row_miou
from labelbox.data.serialization import NDJsonConverter
from labelbox.data.annotation_types import ScalarMetric, LabelList, Label, RasterData, Mask, Point, Rectangle, ObjectAnnotation

* Configure client

In [3]:
# If you don't want to give google access to drive you can skip this cell
# and manually set `API_KEY` below.

COLAB = "google.colab" in str(get_ipython())
if COLAB:
    !pip install colab-env -qU
    from colab_env import envvar_handler
    envvar_handler.envload()

API_KEY = os.environ.get("LABELBOX_API_KEY")
if not os.environ.get("LABELBOX_API_KEY"):
    API_KEY = getpass("Please enter your labelbox api key")
    if COLAB:
        envvar_handler.add_env("LABELBOX_API_KEY", API_KEY)

In [4]:
ENDPOINT = "https://api.labelbox.com/graphql"
client = Client(api_key=API_KEY, endpoint = ENDPOINT)

In [5]:
load_model()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://cloud-tpu-checkpoints/mask-rcnn/1555659850/variables/variables


## Make Inferences

In [6]:
def get_annotations(boxes, classes, seg_masks):
    annotations = []
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        name = class_mappings[class_idx]
        value = None
        classifications = []
        if name in ['person', 'handbag']:
            value = Rectangle(
                start = Point(x = box[1], y = box[0]), end = Point(x = box[3], y = box[2])
            )
        elif name == 'car':
            value = Mask(mask = RasterData.from_2D_arr(arr = seg), color = (1,1,1))            
        if value is not None:
            annotations.append(
                ObjectAnnotation(
                    name = name,
                    value = value
                )
            ) 
    return annotations

### We can just start creating predictions whether or not we have a 
image_urls = ['https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg']
predictions = LabelList([])

for image_url in image_urls:
    image_data = RasterData(url = image_url)
    height, width = image_data.data.shape[:2]
    prediction = predict(np.array([image_data.im_bytes]), min_score=0.5, height=height, width = width)
    annotations = get_annotations(prediction['boxes'], prediction['class_indices'], prediction['seg_masks'])
    predictions.append(Label(
        data = image_data,
        annotations = annotations
    ))
    

## Project Setup

* Create project and attach dataset
* Automatically create an ontology from the predictions

In [7]:
project = client.create_project(name="mea_project")
dataset = client.create_dataset(name="mea_dataset")
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == 'editor'))
# Use the label list to build the ontology
project.setup(editor, predictions.get_ontology().asdict())
project.datasets.connect(dataset)
project.enable_model_assisted_labeling()

True

## Add Required Urls and Ids

In [8]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
predictions.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .assign_schema_ids(OntologyBuilder.from_project(project)) \
         .add_to_dataset(dataset, signer)

1it [00:02,  2.40s/it]
1it [00:00, 8830.11it/s]
1it [00:00, 261.38it/s]


## MAL: Upload Predictions to a Project
* Pre-label image so that we can quickly create ground truth
* This step is not required for MEA.

In [9]:
upload_task = MALPredictionImport.create_from_objects(
    client, 
    project.uid, 
    f'mal-import-{uuid.uuid4()}', 
    list(NDJsonConverter.serialize(predictions))
)
upload_task.wait_until_done()
print(upload_task.state)

AnnotationImportState.FINISHED


In [10]:
upload_task.statuses[:5]

[{'uuid': '3b3ea434-ed2b-455c-a558-f271c0e61f02',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': 'f526b18e-6ef5-4406-8e0e-2b9c6b1461d5',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': '432af859-da16-4983-9d10-7f3453b0fb51',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': 'c9ec723e-c34f-413a-bde5-81f2986018b7',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': '09aef802-ec84-4114-bd59-c88e53cf47a3',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'}]

## Label
* Create ground truth data for MEA
* Click on link below to label

In [11]:
print(f"https://app.labelbox.com/projects/{project.uid}")

https://app.labelbox.com/projects/ckrmmiwe30ejg0y8tgeev6slv


In [12]:
labels = project.export_labels().as_list()

## MEA
1. Create a model
    * Think of this as a model that you want to perform experiments on
2. Create a model run
    * Think of this as a single experiment for a particular model.
    * E.g. this model run is for an instance of a model with particular hyperparameters
3. Select the ground truth annotations for analysis
4. Compute metrics
4. Upload model predictions and metrics to labelbox

In [14]:
model = client.create_model(name = "test-model-12", ontology_id = project.ontology().uid)
model_run = model.create_model_run('test-run-1')

In [15]:
model_run.upsert_labels([label.uid for label in labels])

True

In [16]:
label_lookup = {label.data.uid : label for label in labels}

In [17]:
for pred in predictions:
    label = label_lookup[pred.data.uid]
    score = data_row_miou(label, pred)
    if score is None:
        continue
        
    pred.annotations.append(
        ScalarMetric(value = score)
    )
    

In [18]:
upload_task = model_run.add_predictions(f'mea-import-{uuid.uuid4()}', list(NDJsonConverter.serialize(predictions)))

In [19]:
upload_task.wait_until_done()
print(upload_task.state)

AnnotationImportState.FINISHED


In [20]:
upload_task.statuses

[{'uuid': 'ad91e2bc-052e-4469-a4f8-728cf7ae381e',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': '63754cd5-b592-4767-8527-2a519efef604',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': 'c4ae076e-85de-4339-8e70-1fe7f16d55df',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': '9dd46cd7-0a9c-4089-a7b0-e394f505fbbc',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': '8baed202-e172-4204-8e84-a26e90154882',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': '88a8efe3-6682-4120-8268-700212e7a67a',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': 'b43d1284-3927-46ea-9529-ef1ce6ee15aa',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid': '0b03bcf7-7335-48b5-a6b0-3db9b78c04a0',
  'dataRow': {'id': 'ckrmmj0mb0nnq0zt7c7150ycn'},
  'status': 'SUCCESS'},
 {'uuid'

In [21]:
# View the uploaded predictions and labels (annotation group) in the web app
annotation_group = next(model_run.annotation_groups())
print(annotation_group.url)

https://app.labelbox.com/models/9be7cdc9-3023-066a-1301-7a63c52ff702/9be7cdc9-a7df-012f-ba56-12f894603b29/AllDatarowsSlice/e6e30c8e-1b2c-4478-8304-b9766cec0cb4?view=carousel
